In [0]:
# Google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!pip install mne
!pip install python_speech_features

from sklearn.preprocessing import normalize
from collections import OrderedDict
import numpy as np
import python_speech_features
import tensorflow as tf
from datetime import datetime
import tensorflow as tf
from load_data import BCICompetition4Set2A
import helper_functions as hf
from sklearn.metrics import cohen_kappa_score
import pickle

In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
# Data helper functions
test_subject = 9
test_subject -= 1
r = 9

def BCI_read_data(filename, labels_filename):    
    train_loader = BCICompetition4Set2A(filename=filename, labels_filename=labels_filename)
    train_cnt = train_loader.load()
    train_cnt = train_cnt.drop_channels(['STI 014', 'EOG-left', 'EOG-central', 'EOG-right'])
    assert len(train_cnt.ch_names) == 22
    
    train_cnt = hf.mne_apply(lambda a: a * 1e6, train_cnt)
    train_cnt = hf.mne_apply(lambda a: hf.bandpass_cnt(a, 0, 38.0, train_cnt.info['sfreq'],
                               filt_order=3,
                               axis=1), train_cnt)
    train_cnt = hf.mne_apply(lambda a: hf.exponential_running_standardize(a.T, factor_new=1e-3,
                                                  init_block_size=1000,
                                                  eps=1e-4).T, train_cnt)
    
    return train_cnt

def BCI_load_all_data():
    ival = [-500, 4000]
    marker_def = OrderedDict([('Left Hand', [1]), ('Right Hand', [2],),
                              ('Foot', [3]), ('Tongue', [4])])
    
    raw_files = [BCI_read_data("A0" + str(f) + "T.gdf", 
                               "A0" + str(f) + "T.mat") for f in range(1, r+1)]
    
    
    train_set_list = [hf.create_signal_target_from_raw_mne(raw, marker_def, ival) for raw in raw_files]
    train_set_X = np.concatenate([train_set_list[i].X for i in  range(r) if i!=test_subject])
    # train_set_X = np.concatenate([train_set.X for train_set in train_set_list[:-1]])
    train_set_y = np.concatenate([train_set_list[i].y for i in  range(r) if i!=test_subject])
    # train_set_y = np.concatenate([train_set.y for train_set in train_set_list[:-1]])
    train_set_z = np.concatenate([np.zeros((288)) + i for i in range(r-1)])
    
    train_set_X = np.stack([np.stack([np.array(python_speech_features.mfcc(channel, 250, numcep=25)) for channel in event]) for event in train_set_X])
    
    indices = np.arange(train_set_y.shape[0])
    np.random.shuffle(indices)
    
    train_set = hf.SignalAndTarget(train_set_X[indices, :, :], train_set_y[indices], train_set_z[indices])
    train_set, valid_set = hf.split_into_two_sets(train_set, first_set_fraction=1-0.2)
    
    test_set_X = train_set_list[test_subject].X
    test_set_y = train_set_list[test_subject].y
    test_set_X = np.stack([np.stack([np.array(python_speech_features.mfcc(channel, 250, numcep=25)) for channel in event]) for event in test_set_X])
    test_set = hf.SignalAndTarget(test_set_X, test_set_y)
    
    iterator = hf.CropsFromTrialsIterator(batch_size=60, input_time_length=13, n_preds_per_input=4)
    
    return iterator, train_set, valid_set, test_set

In [0]:
iterator, train_set, valid_set, test_set = BCI_load_all_data()


In [0]:
# iterator = hf.CropsFromTrialsIterator(batch_size=60, input_time_length=374, n_preds_per_input=4)
# train_set = np.load("/content/gdrive/My Drive/data/train_set_374_13.npy").item(0)
# valid_set = np.load("/content/gdrive/My Drive/data/valid_set_374_13.npy").item(0)
# test_set = np.load("/content/gdrive/My Drive/data/test_set_374_13.npy").item(0)

In [0]:
def shallow_brain_model(features, mode):
    conv1 = tf.layers.conv2d(inputs=features, filters = 20, kernel_size=(1, 13), strides=(1, 3), activation=None)
    batch1 = tf.layers.batch_normalization(conv1, momentum=0.1)
    activation1 = tf.nn.relu(batch1)
    dropout1 = tf.layers.dropout(activation1, 0.5, training=mode)
    
    conv2 = tf.layers.conv2d(inputs=dropout1, filters = 10, kernel_size=(8, 2), strides=(3, 1), activation=None)
    batch2 = tf.layers.batch_normalization(conv2, momentum=0.1)
    activation2 = tf.nn.relu(batch2)
    dropout2 = tf.layers.dropout(activation2, 0.5, training=mode)
    
    conv3 = tf.layers.conv2d(inputs=dropout2, filters = 8, kernel_size=(1, 15), strides=4, activation=None)
    batch3 = tf.layers.batch_normalization(conv3, momentum=0.1)
    activation3 = tf.nn.relu(batch3)
    
    conv4 = tf.layers.conv2d(inputs=activation3, filters = 4, kernel_size=(2, 27), strides=1, activation=None)
    
    logits = tf.reshape(conv4, [-1, 4])

    return logits
  
def detect_subject_model(features, mode):
    conv1 = tf.layers.conv2d(inputs=features, filters = 20, kernel_size=(1, 13), strides=(1, 3), activation=None)
    batch1 = tf.layers.batch_normalization(conv1, momentum=0.1)
    activation1 = tf.nn.relu(batch1)
    dropout1 = tf.layers.dropout(activation1, 0.5, training=mode)
    
    conv2 = tf.layers.conv2d(inputs=dropout1, filters = 10, kernel_size=(8, 2), strides=(3, 1), activation=None)
    batch2 = tf.layers.batch_normalization(conv2, momentum=0.1)
    activation2 = tf.nn.relu(batch2)
    dropout2 = tf.layers.dropout(activation2, 0.5, training=mode)
    
    conv3 = tf.layers.conv2d(inputs=dropout2, filters = 8, kernel_size=(1, 15), strides=4, activation=None)
    batch3 = tf.layers.batch_normalization(conv3, momentum=0.1)
    activation3 = tf.nn.relu(batch3)
    
    conv4 = tf.layers.conv2d(inputs=activation3, filters = 8, kernel_size=(2, 27), strides=1, activation=None)
    
    logits = tf.reshape(conv4, [-1, 8])

    return logits

def mask_model(features, mode):
    conv1 = tf.layers.conv2d(inputs=features, filters=7, kernel_size=(15, 23), strides=(1, 3), activation=None)
    batch1 = tf.layers.batch_normalization(conv1, momentum=0.1)
    activation1 = tf.nn.elu(batch1)
    dropout1 = tf.layers.dropout(activation1, 0.5, training=mode)
    
    conv2 = tf.layers.conv2d_transpose(inputs=dropout1, filters=13, kernel_size=(15, 23), strides=(1, 3), activation=None)
    batch2 = tf.layers.batch_normalization(conv2, momentum=0.1)
    activation2 = tf.nn.elu(batch2)
    
    return activation2

In [0]:
tf.reset_default_graph()

# MFCC_features = tf.placeholder(tf.float32, shape=[60, 22, 374, 13])
MFCC_features = tf.placeholder(tf.float32, shape=[None, None, None, 25])
# mask = tf.placeholder(tf.float32, shape=[None, None, None, 13])
labels = tf.placeholder(tf.int32, shape=[None])
lr = tf.placeholder(tf.float32)
mode = tf.placeholder(tf.bool)

m_model = mask_model(MFCC_features, mode)
subject_model = detect_subject_model(m_model, mode)
class_model = shallow_brain_model(MFCC_features, mode)

if tf.train.get_global_step() is None:
    tf.train.create_global_step()
    
print(np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()]))

In [0]:
subject_loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=subject_model)
class_loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=class_model)

subject_optimizer = tf.train.AdamOptimizer(learning_rate=lr)
class_optimizer = tf.train.AdamOptimizer(learning_rate=lr)

subject_op = subject_optimizer.minimize(loss=subject_loss, global_step=tf.train.get_global_step())
class_op = class_optimizer.minimize(loss=class_loss)

In [0]:
epoch_number = 600

with tf.Session() as sess:
  
    sess.run(tf.global_variables_initializer())
    
    print("{} Start training...".format(datetime.now()))
    
    for epoch in range(epoch_number):
      
        all_pred_subject = []
        all_loss_subject = []
        all_subjects = []
        val_pred_subject = []
        val_loss_subject = []
        val_subjects = []
        
        
        all_pred_class = []
        all_loss_class = []
        all_targets = []
        val_pred_class = []
        val_loss_class = []
        val_targets = []
        test_pred_class = []
        test_targets = []
      
        batch_generator = iterator.get_batches(train_set, shuffle=True)
        batch_generator_2 = iterator.get_batches(valid_set, shuffle=True)
        batch_generator_3 = iterator.get_batches(test_set, shuffle=True)
        
        for inputs, targets, subjects in batch_generator:
#             _, s_loss, subject_logits = sess.run([subject_op, subject_loss, subject_model], feed_dict={MFCC_features: inputs, mode:True, lr:0.0001, labels:subjects})
            
#             prediction = np.argmax(subject_logits, 1)
#             all_pred_subject.extend(prediction)
#             all_loss_subject.append(s_loss)
#             all_subjects.extend(subjects)

            _, c_loss, class_logits = sess.run([class_op, class_loss, class_model], feed_dict={MFCC_features: inputs, mode:True, lr:0.001, labels:targets})
            
            prediction = np.argmax(class_logits, 1)
            all_pred_class.extend(prediction)
            all_loss_class.append(c_loss)
            all_targets.extend(targets)
            
            
#         subject_accuracy = np.sum(np.equal(all_pred_subject, all_subjects)) / float(len(all_pred_subject))
        class_accuracy = np.sum(np.equal(all_pred_class, all_targets)) / float(len(all_pred_class))
        
        num_nodes = len([n.name for n in tf.get_default_graph().as_graph_def().node])

        print("Epoch {} nodes {}".format(epoch, num_nodes))
#         print("Subjects: Loss {:.6f}, Accuracy: {}, Kappa {}".format(np.mean(all_loss_subject), subject_accuracy, cohen_kappa_score(all_pred_subject, all_subjects)))
        print("Class:    Loss {:.6f}, Accuracy: {}, Kappa {}".format(np.mean(all_loss_class), class_accuracy, cohen_kappa_score(all_pred_class, all_targets)))

        
        
        
        
        
        for inputs, targets, subjects in batch_generator_2:
#             s_loss, subjects_logits = sess.run([subject_loss, subject_model], feed_dict={MFCC_features: inputs, mode:False, labels:subjects})
            
#             prediction = np.argmax(subjects_logits, 1)
#             val_pred_subject.extend(prediction)
#             val_loss_subject.append(s_loss)
#             val_subjects.extend(subjects)
            
            
            c_loss, class_logits = sess.run([class_loss, class_model], feed_dict={MFCC_features: inputs, mode:False, labels:targets})

            prediction = np.argmax(class_logits, 1)
            val_pred_class.extend(prediction)
            val_loss_class.append(c_loss)
            val_targets.extend(targets)

        class_accuracy = np.sum(np.equal(val_pred_class, val_targets)) / float(len(val_pred_class))
#         subject_accuracy = np.sum(np.equal(val_pred_subject, val_subjects)) / float(len(val_pred_subject))

          
        print("Class:    Loss {:.6f}, Accuracy: {}, Kappa: {}".format(np.mean(val_loss_class), class_accuracy, cohen_kappa_score(val_pred_class, val_targets)))
#         print("Subjects: Loss {:.6f}, Accuracy: {}, Kappa: {}".format(np.mean(val_loss_subject), subject_accuracy, cohen_kappa_score(val_pred_subject, val_subjects)))
        
        
        
        
        
        
#         for inputs, targets, subjects in batch_generator_3:
#             c_loss, class_logits = sess.run([class_loss, class_model], feed_dict={MFCC_features: inputs, mode:False, labels:targets})

#             prediction = np.argmax(class_logits, 1)
#             test_pred_class.extend(prediction)
#             test_targets.extend(targets)

#         class_accuracy = np.sum(np.equal(test_pred_class, test_targets)) / float(len(test_pred_class))

#         print("TestC:    Loss {:.6f}, Accuracy: {}, Kappa: {}".format(c_loss, class_accuracy, cohen_kappa_score(test_pred_class, test_targets)))
